In [ ]:
import numpy as np 

def compute_chi(data):
    '''
    Computes the value of Chi, a synchrony measurement that compares the variance of individual voltage traces and the variance of the mean of the voltages.

    Parameters:
        data (list[float]):
            Voltage traces with each row corresponding one neuron.

    Returns:
        chi (float):
            The value of Chi.
    '''

    #calculate the average voltage as a function of time
    mean_voltage = np.mean(data,axis=0)

    #calculate the variance of each trave and the average voltage
    ind_variance = np.mean(np.square(data),axis=1) - np.mean(data,axis=1)**2
    total_variance = np.mean(np.square(mean_voltage)) - np.mean(mean_voltage)**2

    #calculate chi
    chi = np.sqrt(total_variance**2 / np.mean(ind_variance**2))

    return chi 


In [ ]:
import numpy as np 
from scipy.sparse import dok_matrix
import scipy as sp
from scipy import signal

def compute_van_Rossum_distance(spike_matrix,t,t_R,traces = False):
    '''
    Computes the Van Rossum Distance, a synchrony measurement that computes the area underneath the curve that is the difference between the convolved spike trains of two neurons.

    Parameters:
        spike_matrix (tuple[tuple[int,int]] | sparse_matrix):
            matrix containing spike trains, each row contains a diffent neuron.
        t (list[float]):
            time array, time points of the simulation
        t_R (float):
            Time constant
        traces (bool, optional):
            False, default: the function does not return the convolved spike trains
            True: the function returns the convolved spike trains

    Returns:
        van_Rossum (float):
            The computed value of Van Rossum distance
        waveforms[0,:] (tuple[float]):
            The convolved spike train of the first neuron.
        waveforms[1,:] (tuple[float]):
            The convoled spike train of the second neuron.
    '''

    #compute the time step of the simulation
    dt = (t[len(t)-1] - t[0] ) / len(t) 

    #we need to work with a np.ndarray for the convolution, if it is a sparse_matrix, change it to that type
    if type(spike_matrix) is not np.ndarray:
        spike_matrix = np.array(spike_matrix.todense())

    #construct kernel and prepare the spike trains to be convolved
    N = len(spike_matrix[:,0])  
    van_Rossum = np.zeros((N,N))
    kernel = np.exp(-t/t_R)
    test = signal.convolve(spike_matrix[0,:],kernel)[0:len(spike_matrix[0,:])]
    waveforms = np.zeros((N,len(test)))

    #Convolve spike trains with kernel (2D convolution iwth 1 as column convolution, i.e. no convolution)
    for j in range(0,N):
        waveforms[j,:] = signal.convolve(spike_matrix[j,:], kernel)[0:len(spike_matrix[j,:])]

    #compute van Rossum distance between each pair of spike trains
    for j in range(0,N):
        waveform_difference = waveforms - waveforms[j,:]
        van_Rossum[j,:] = dt*np.sum(np.square(waveform_difference),axis=1)/t_R

    if traces:
        return van_Rossum, waveforms[0,:], waveforms[1,:]
    else:
        return van_Rossum

def compute_van_Rossum_distance_2(spike_matrix,t,t_R):
    '''
    Computes the Van Rossum Distance, a synchrony measurement that computes the area underneath the curve that is the difference between the convolved spike trains of two neurons.

    Parameters:
        spike_matrix (tuple[tuple[int,int]] | sparse_matrix):
            matrix containing spike trains, each row contains a diffent neuron.
        t (list[float]):
            time array, time points of the simulation
        t_R (float):
            Time constant

    Returns:
        van_Rossum (float):
            The computed value of Van Rossum distance
    '''

    #compute the time step of the simulation
    dt = (t[len(t)-1] - t[0] ) / len(t) 

    #we need to work with a np.ndarray for the convolution, if it is a sparse_matrix, change it to that type
    if type(spike_matrix) is not np.ndarray:
        spike_matrix = np.array(spike_matrix.todense())

    #construct kernel and prepare the spike trains to be convolved
    N = len(spike_matrix[:,0])  
    van_Rossum = np.zeros((N,N))
    kernel = np.exp(-t/t_R)
    test = signal.convolve(spike_matrix[0,:],kernel)[0:len(spike_matrix[0,:])]
    waveforms = np.zeros((N,len(test)))

    #Convolve spike trains with kernel (2D convolution iwth 1 as column convolution, i.e. no convolution)
    for j in range(0,N):
        waveforms[j,:] = signal.convolve(spike_matrix[j,:], kernel)[0:len(spike_matrix[j,:])]

    #compute van Rossum distance between each pair of spike trains
    for j in range(0,N):
        waveform_difference = waveforms - waveforms[j,:]
        van_Rossum[j,:] = np.sqrt(np.trapz(np.square(waveform_difference)/t_R,dx=dt))
    return van_Rossum


In [ ]:
from scipy import signal
def compute_Reliability(spike_matrix,t,t_R):
    '''
    Computes the value of Reliability, a synchrony measurement that computes the variance of a convoluted spike train that is the sum of the spike trains of each neuron.

    Parameters:
        spike_matrix (tuple[tuple[int,int]] | sparse_matrix):
            matrix containing spike trains, each row contains a diffent neuron.
        t (list[float]):
            time array, time points of the simulation
        t_R (float):
            Time constant

    Returns:
        reliability / reliability_max (float):
            The computed value of reliability, normalised so (theorically) is between 0 and 1.
        Convolved_matrix (tuple[float]):
            The convolved spike train.
    '''

    #compute the time step of the simulation
    T = t[-1] - t[0]
    dt = T / len(t)

    #we need to work with a np.ndarray for the convolution, if it is a sparse_matrix, change it to that type
    if type(spike_matrix) is not np.ndarray:
        spike_matrix = np.array(spike_matrix.todense())

    #Compute the mean number of spikes
    num_spikes = (spike_matrix>0).sum()/num_neurons

    #Compute the kernel, and convolve the sum spike train with it
    joined_matrix = np.sum(spike_matrix,axis=0)
    kernel = 1/ t_R *np.exp(-t / t_R)
    Convolved_matrix = signal.convolve(joined_matrix,kernel)[0:len(spike_matrix[0,:])]
    #Convolved_matrix = joined_matrix

    #compute the measurements of reliability
    reliability = 1 / t[len(t) - 1] * np.trapz(np.square(Convolved_matrix), dx = dt) - np.square( 1 / t[len(t) - 1] * np.trapz(Convolved_matrix,dx =dt))
    reliability_max = ( num_neurons**2 * num_spikes / ( t_R * 2 * T ) - ( num_neurons * num_spikes / T )**2 )
    #reliability_max = 1
    return reliability/reliability_max, Convolved_matrix